In [ ]:
''' TODO:
0. Implement filtering on Database level, no need to fetch all densities and filter them in the class.
1. Send telegram notification if found density: < 2% and > 60 mins old (or 120 mins)
2. Track how many times asset price approached the density (visinity = 1%, first touch, second touch, etc.)

'''

' TODO\n1. Send telegram notification if found density: < 2% and > 60 mins old (or 120 mins)\n2. Implement method that only tracks without displaying part!\n\n'

In [1]:
# Import required libraries
from sqlalchemy import Column, Float, String, DateTime, Integer, create_engine, UniqueConstraint
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import sessionmaker

# Required imports
import ccxt
import time
from datetime import datetime

from rich.console import Console
from rich.table import Table
from rich.style import Style
from IPython.display import clear_output

In [3]:
# Cell 1: DB manager

# Create base class for declarative models
Base = declarative_base()

class Density(Base):
    """SQLAlchemy model for storing order book densities"""
    __tablename__ = 'densities'
    
    pair = Column(String, primary_key=True, nullable=False)
    price = Column(Float, primary_key=True, nullable=False)
    side = Column(String, nullable=False)
    size = Column(Float, nullable=False)
    worth = Column(Float, nullable=False)
    spread_price = Column(Float, nullable=False)
    timestamp = Column(DateTime, default=datetime.utcnow)
    
    __table_args__ = (
        UniqueConstraint('pair', 'price', name='uix_pair_price'),
    )
    
    def __repr__(self):
        return f"<Density(pair={self.pair}, side={self.side}, price={self.price}, "
        f"size={self.size}, worth={self.worth}, spread_price={self.spread_price}, time={self.timestamp})>"

class DatabaseManager:
    def __init__(self, session_factory):
        self.Session = session_factory

    def fetch_densities(self, pair=None):
        """Fetch all densities for a specific pair."""
        session = self.Session()
        try:
            condition = Density.pair == pair if pair else True
            densities = session.query(Density).filter(condition).order_by(Density.pair.asc(), Density.price.desc()).all()
            # densities = query.order_by(Density.pair.asc(), Density.price.desc()).all()
            return densities
        except Exception as e:
            print(f"Error fetching densities: {e}")
            return []
        finally:
            session.close()

    def update_density(self, density, side=None, size=None, worth=None, spread_price=None):
        """Update an existing density."""
        session = self.Session()
        existing_density = session.query(Density).filter_by(pair=density.pair, side=density.side, price=density.price).first()
        try:
            if side is not None:
                existing_density.side = side
            if size is not None:
                existing_density.size = size
            if worth is not None:
                existing_density.worth = worth
            if spread_price is not None:
                existing_density.spread_price = spread_price
            session.commit()
        except Exception as e:
            session.rollback()
            print(f"Error updating density: {e}")
        finally:
            session.close()

    def delete_density(self, density):
        """Delete a density."""
        session = self.Session()
        try:
            session.delete(density)
            session.commit()
        except Exception as e:
            session.rollback()
            print(f"Error deleting density: {e}")
        finally:
            session.close()

    def add_density(self, pair, side, price, size, worth, spread_price):
        """Add a new density to the database."""
        session = self.Session()
        try:
            density = Density(
                pair=pair,
                side=side,
                price=price,
                size=size,
                worth=worth,
                spread_price=spread_price,
                timestamp=datetime.utcnow()
            )
            session.add(density)
            session.commit()
        except Exception as e:
            session.rollback()
            print(f"Error adding density: {e}")
        finally:
            session.close()


In [5]:
# Create db_manager

engine = create_engine("sqlite:///densities.db")  # Replace with your actual database URI
Session = sessionmaker(bind=engine)
db_manager = DatabaseManager(session_factory=Session)

In [7]:
# Cell 3 (Main Tracking Class):

class BinanceDensityTracker:
    def __init__(self, exchange, db_manager, value_thresholds, display_price_threshold=0.05, display_detected_threshold=10):
        self.exchange = exchange
        self.db_manager = db_manager
        self.value_thresholds = value_thresholds
        self.display_price_threshold = display_price_threshold
        self.display_detected_threshold = display_detected_threshold
        # self.price_range_threshold = price_range_threshold  # obsolete - used before I started deeper Order Book fetching 

    def _process_pair_density(self, pair):
        """Process densities for a single pair."""
        # session = self.db_manager.Session()
        value_threshold = self.value_thresholds.get(pair.split('/')[0], 
                                                    self.value_thresholds['default'])

        # Step 1: Fetch order book
        # print(f"calling {pair}")
        try:
            order_book = self.exchange.fetch_order_book(pair, limit=1000)
        except Exception as e:
            print(f"Error {e} occured while fetching Order Book for {pair}")
            return
        asks = order_book['asks']
        bids = order_book['bids']

        max_ask_price = asks[-1][0] if asks else float('inf')
        min_ask_price = asks[0][0] if asks else 0
        max_bid_price = bids[0][0] if bids else float('inf')
        min_bid_price = bids[-1][0] if bids else 0
        # print(f"max_ask: {max_ask_price}, min_ask: {min_ask_price}, max_bid: {max_bid_price}, min_bid: {min_bid_price}")

        # Step 2: Identify new densities in the order book
        new_densities = []
        for price, size in asks:
            worth = price * size
            if worth > value_threshold:
                new_densities.append({
                    "pair": pair,
                    "side": 'Ask',
                    "price": price,
                    "size": size,
                    "worth": worth,
                    "spread_price": min_ask_price
                })

        for price, size in bids:
            worth = price * size
            if worth > value_threshold:
                new_densities.append({
                    "pair": pair,
                    "side": 'Bid',
                    "price": price,
                    "size": size,
                    "worth": worth,
                    "spread_price": max_bid_price
                })

        # Step 3: Fetch old densities for the pair from DB
        old_densities = self.db_manager.fetch_densities(pair)

        # Step 4: Compare old densities with new densities
        for old_density in old_densities:
            old_price = old_density.price
            old_side = old_density.side
            old_worth = old_density.worth
            old_spread_price = old_density.spread_price
            
            # (a) Update existing densities
            matching_new_density = next(
                (d for d in new_densities 
                 if d["price"] == old_price and d["side"] == old_side), 
                None
            )
            if matching_new_density:
                self.db_manager.update_density(
                    old_density,
                    matching_new_density["side"],
                    matching_new_density["size"],
                    matching_new_density["worth"],
                    matching_new_density["spread_price"]
                )
                new_densities.remove(matching_new_density)
                continue

            '''
            Removed the obolete code, as Order Book limit=1000 fetches much futher than 10% price_ragne_threshold
            '''
            
            # (c) Remove densities not matching any condition
            self.db_manager.delete_density(old_density)

        # Add new densities that didn't match any old density
        for new_density in new_densities:
            self.db_manager.add_density(
                pair=new_density["pair"],
                side=new_density["side"],
                price=new_density["price"],
                size=new_density["size"],
                worth=new_density["worth"],
                spread_price=new_density["spread_price"]
            )

        # session.close()

    def display_all_densities(self):
        """Display all densities in a table."""

        # Define background styles for highlighting
        ask_highlight_style = Style(bgcolor="#ffe4e1")  # Light red background for "Ask"
        bid_highlight_style = Style(bgcolor="#f0fff0")  # Light green background for "Bid"

        '''
        TODO: Highlight densities < 1.5% and older than 30 mins
        '''
        
        display_price_threshold = self.display_price_threshold   # show only debsities within 5% from current price
        display_detected_threshold = self.display_detected_threshold  # minimum minutes to show the density

        # session = self.db_manager.Session()
        try:
            # densities = session.query(Density).order_by(Density.pair.asc(), Density.price.desc()).all()
            densities = self.db_manager.fetch_densities()
            # Create a Rich table
            table = Table(title="Spot and Futures Densities", show_header=True, header_style="bold magenta")
            
            # Add columns to the table
            table.add_column("Pair", justify="center")
            table.add_column("Side", justify="center")
            table.add_column("Price", justify="right")
            table.add_column("Size", justify="right")
            table.add_column("Worth ($)", justify="right")
            table.add_column("Distance (%)", justify="right")
            table.add_column("Detected", justify="right")
            
            # Add rows to the table
            for density in densities:
                distance = (density.price - density.spread_price) / density.spread_price
                # Show only rows that are not very far, e.g., 5% distance from the price
                if abs(distance) <= display_price_threshold:
                
                    # Calculate the relative time for the "detected" column
                    time_difference = datetime.utcnow() - density.timestamp
                    total_minutes = int(time_difference.total_seconds() // 60)
                    # Show only densities that are detected at least 10 mins ago
                    if total_minutes >= display_detected_threshold:
                        hours, minutes = divmod(total_minutes, 60)
                        
                        if hours > 0:
                            relative_time = f"{hours}h {minutes}m" if minutes > 0 else f"{hours}h"
                        else:
                            relative_time = f"{minutes}m" if minutes > 0 else "now"
                    
                        row_style = None
                        # Highlight rows that are older than 30 minutes
                        highlight = total_minutes >= 30
                        if highlight:
                            row_style = ask_highlight_style if density.side == "Ask" else bid_highlight_style
        
                        # Add row to the table with appropriate styles
                        table.add_row(
                            density.pair,  # Pair
                            density.side,  # Side
                            f"{density.price:,.4f}",  # Price
                            f"{density.size:,.0f}",  # Size
                            f"{int(round(density.worth / 50000) * 50000 // 1000)}k",  # Worth
                            f"{distance * 100:,.2f}%",  # Distance
                            relative_time,  # Relative Detected time
                            style=row_style,
                        )            
            # Display the Rich table
            console.print(table)
        except Exception as e:
            print(f"Error displaying densities: {e}")
        finally:
            # session.close()
            pass

    def send_telegram_alert():
        densities = self.db_manager.fetch_densities()
        for density in densities:
            if density.detected > "older_than_two_hours" and abs(density.spread_price - density.price) / density.price < 0.01:
                # TODO: send_alert_to_telegram_channel
                pass
    
    def run(self, pairs, display=True, telegram_alert=False):
        """Process densities for all pairs and display results."""
        try:
            while True:  # Infinite loop to fetch data repeatedly                
                if display:
                    # Display all densities           
                    # Clear previous output in the notebook cell
                    clear_output(wait=True)
                    time.sleep(1)
                    self.display_all_densities()
                for pair in pairs:
                    self._process_pair_density(pair)
                if telegram_alert:
                    # TODO: send telegram alerts
                    self.seld_telegram_alert()
                    pass
        except KeyboardInterrupt:
            print("\nProcess interrupted. Exiting...")

In [9]:
# Cell 2 (Configuration):

# Initialize the Binance exchange instance
exchange = ccxt.binance({
    'enableRateLimit': True,  # Enable rate limiting
})

# Create a console instance
console = Console()

In [57]:
# Configuration
symbols = ['ADA','AAVE','ALGO','ALICE','ASTR','AMB','APT','AR','ARB','AVAX','ACX','BCH','BONK','BNB','BNX',
           'BTC','CELO','CFX','COMBO','DYDX','EGLD','ETH','DEFI','DENT','DOGE','FET','FIL','FLOW',
           'GMT','GRT','HBAR','HOOK','ICP','INJ','IOTA','JTO',
           'KAVA','LEVER','LISTA','LDO','LINK','MASK','MINA','NEAR','ONDO','PEOPLE','PAXG','RENDER','RSR','RPL',
           'SAND','1000SATS','SHIB','SOL','SUI','TAO','TON','TRX','VET','WLD','XAI','XRP','XLM','ZRO']

symbols_1000_futures = ['BONK','SHIB']
symbols_no_futures = ['PAXG']

pairs = []
for symbol in symbols:
    pairs.append(f"{symbol}/USDT")  # Always add spot pair
    if symbol in symbols_1000_futures:
        pairs.append(f"1000{symbol}/USDT:USDT")  # Special futures format
    elif symbol in symbols_no_futures:
        pass
    else:
        pairs.append(f"{symbol}/USDT:USDT")  # Standard futures format

value_thresholds = {
    'default': 200000,
    **{k: 5000000 for k in ['1000SATS']},
    **{k: 1000000 for k in ['DOGE','BTC','ETH','BNB','MASK','XRP','SOL']},
    **{k: 300000 for k in ['ADA','LINK','TRX']},
    **{k: 500000 for k in ['HBAR','SUI']}
}

In [59]:
# Cell 4: create tracker object

# Create and start the tracker
tracker = BinanceDensityTracker(
    exchange=exchange,
    db_manager=db_manager,
    value_thresholds=value_thresholds,
    display_price_threshold=0.03,                   # only show density within +/-5% price range
    display_detected_threshold=5                   # only display density detected earlier than 10 mins ago
)

In [61]:
# Track densities

tracker.run(pairs, display=True, telegram_alert=False)

                                  Spot and Futures Densities                                   
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃        Pair        ┃ Side ┃    Price ┃           Size ┃ Worth ($) ┃ Distance (%) ┃ Detected ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ 1000SATS/USDT:USDT │ Bid  │   0.0001 │ 49,966,911,399 │     6050k │       -1.39% │      58m │
│ 1000SATS/USDT:USDT │ Bid  │   0.0001 │ 46,723,605,793 │     5600k │       -1.80% │      58m │
│ 1000SATS/USDT:USDT │ Bid  │   0.0001 │ 84,799,649,591 │    10200k │       -1.96% │      58m │
│ 1000SATS/USDT:USDT │ Bid  │   0.0001 │ 70,594,466,395 │     8450k │       -2.04% │      58m │
│ 1000SATS/USDT:USDT │ Bid  │   0.0001 │ 50,622,046,772 │     6050k │       -2.78% │      58m │
│ 1000SATS/USDT:USDT │ Bid  │   0.0001 │ 46,977,017,648 │     5600k │       -2.86% │      58m │
│   APT/USDT:USDT    │ Bid  │   5.8199 │         37,721 │      200k │       -1.23% │       6m │
│     AVAX/USDT      │ Bid  │  25.0400 │         11,213 │      300k │       -0.56% │       6m │
│   BNB/USDT:USDT    │ Ask  │ 588.2000 │          2,000 │     1200k │        0.59% │   1h 37m │
│   LINK/USDT:USDT   │ Ask  │  19.2410 │         26,571 │      500k │        1.30% │      15m │
│   LINK/USDT:USDT   │ Bid  │  18.8100 │         27,256 │      500k │       -0.96% │       5m │
│  LISTA/USDT:USDT   │ Bid  │   0.1680 │      1,191,445 │      200k │       -2.78% │    1h 1m │
│   MASK/USDT:USDT   │ Bid  │   2.1510 │      1,841,188 │     3950k │       -1.64% │    1h 1m │
│   MASK/USDT:USDT   │ Bid  │   2.1500 │      3,677,137 │     7900k │       -1.68% │    1h 1m │
│   MASK/USDT:USDT   │ Bid  │   2.1497 │        921,058 │     2000k │       -1.70% │    1h 1m │
│   MASK/USDT:USDT   │ Bid  │   2.1450 │      1,846,164 │     3950k │       -1.91% │    1h 1m │
│   MASK/USDT:USDT   │ Bid  │   2.1430 │        746,665 │     1600k │       -2.00% │    1h 1m │
│   MASK/USDT:USDT   │ Bid  │   2.1420 │      1,401,061 │     3000k │       -2.05% │    1h 1m │
│   MASK/USDT:USDT   │ Bid  │   2.1410 │        924,823 │     2000k │       -2.09% │    1h 1m │
│     ONDO/USDT      │ Bid  │   1.4010 │        391,522 │      550k │       -0.47% │      15m │
│   ONDO/USDT:USDT   │ Bid  │   1.4010 │        391,522 │      550k │       -0.46% │      15m │
│   SAND/USDT:USDT   │ Ask  │   0.3944 │      1,108,946 │      450k │        2.41% │   1h 35m │
│      SOL/USDT      │ Ask  │ 200.0000 │          7,267 │     1450k │        0.87% │      19m │
│   TRX/USDT:USDT    │ Ask  │   0.2369 │      1,726,232 │      400k │        1.49% │      34m │
│   TRX/USDT:USDT    │ Bid  │   0.2316 │      1,728,356 │      400k │       -0.77% │       6m │
│   XRP/USDT:USDT    │ Ask  │   2.5022 │        801,812 │     2000k │        0.89% │      14m │
└────────────────────┴──────┴──────────┴────────────────┴───────────┴──────────────┴──────────┘


Process interrupted. Exiting...


### Scratches part (saved old code)

In [ ]:
            '''
            This part is obolete now, as Order Book limit=1000 fetches much futher than 10% price_ragne_threshold
            # (b) Keep old densities in the price range (10%)
            in_price_range = (
                (old_price > max_ask_price and old_price < min_ask_price * (1 + self.price_range_threshold) and old_side == 'Ask') or
                (old_price < min_bid_price and old_price > max_bid_price * (1 - self.price_range_threshold) and old_side == 'Bid')
            )
            if in_price_range and old_worth >= value_threshold:
                new_spread_price = min_ask_price if old_side == 'Ask' else max_bid_price
                # print(f"Updating spread_price from {old_density.spread_price} to {new_spread_price} for {old_density.pair}")
                self.db_manager.update_density(
                    old_density,
                    size=old_density.size,
                    worth=old_density.worth,
                    spread_price=new_spread_price
                )
                continue
            '''

### Testing / Sandbox

In [77]:
symbols = ["LINK/USDT","ADA/USDT","ALGO/USDT","APT/USDT","FIL/USDT","GRT/USDT"]
try:
    order_book = exchange.fetch_order_book(symbols)
except ccxt.BaseError as e:
    print(f"An error occurred: {e}")    

TypeError: can only concatenate str (not "list") to str

In [83]:
symbols = ["LINK/USDT","ADA/USDT","ALGO/USDT","APT/USDT","FIL/USDT","GRT/USDT"]

try:
    exchange.load_markets()
    # Fetch the order book for the specified symbol
    print("Order Book price range")
    for symbol in symbols:
        order_book = exchange.fetch_order_book(symbol, limit=1000000)
    
        # Extract the top 100 asks and bids
        asks = order_book['asks'][::-1]  # Reverse order for descending asks
        bids = order_book['bids']
    
        # Determine the max bid and min ask prices
        max_bid_price = bids[0][0] if bids else 0
        min_bid_price = bids[-1][0]
        neg_dev = (max_bid_price  - min_bid_price) / max_bid_price
        
        min_ask_price = asks[-1][0] if asks else float('inf')
        max_ask_price = asks[0][0]
        pos_dev = (max_ask_price - min_ask_price) / min_ask_price

        print(f"{symbol}: from {min_bid_price} to {max_ask_price}")
        # print(f"{symbol}: from -{neg_dev:.3f} to +{pos_dev:.3f}")
    
except ccxt.BaseError as e:
    print(f"An error occurred: {e}")

Order Book price range
LINK/USDT: from 0.0001 to 72.72
ADA/USDT: from 0.3733 to 1.4449
ALGO/USDT: from 0.0189 to 1.135
APT/USDT: from 0.95 to 99.4
FIL/USDT: from 0.564 to 10.864
GRT/USDT: from 0.0116 to 0.9527
